In [29]:
using Revise

In [30]:
using ModelVerification

In [31]:
using LazySets
using PyCall
using CSV
using ONNX
using Flux
using Test
using NNlib
using ONNXNaiveNASflux
using NaiveNASflux
using Zygote
# using DataFrames
# import Flux: flatten

In [32]:
# using Flux: onehotbatch, onecold, flatten
# using Flux.Losses: logitcrossentropy
# using Statistics: mean
using CUDA
using MLDatasets: CIFAR10, MNIST
using MLUtils: splitobs, DataLoader
using Accessors
using Profile
using LinearAlgebra
using Einsum
using Statistics

In [33]:
struct Layer
    a
    b
end
Flux.@functor Layer (a, b,)

function (f::Layer)(x)
    return NNlib.batched_mul(f.a, x) .+ f.b
end
a = cu(zeros(2, 1, 2))
b = cu(zeros(2, 1, 2))
x = cu([2;;; 4])
y = cu([2; 4;;; 2; 4])
model = fmap(cu, Layer(a, b))
#println(Flux.params(model))
loss(x, y) = Flux.mse(x, y)
optimizer = Flux.Optimiser(Flux.ADAM(0.1))
opt_state = Flux.setup(optimizer, model)
for i in 1 : 5
    losses, grads = Flux.withgradient(model) do m
        result = m(x) 
        loss(result, y)
    end
    Flux.update!(opt_state, model, grads[1])
    println(grads[1])
    println(losses)
    println(Flux.params(model)) 
end


(a = [-2.0; -4.0;;; -4.0; -8.0], b = [-1.0; -2.0;;; -1.0; -2.0])
10.0
Params([[0.1; 0.1;;; 0.1; 0.1], [0.1; 0.1;;; 0.1; 0.1]])
(a = [-1.7; -3.7;;; -3.0; -7.0], b = [-0.85; -1.85;;; -0.75; -1.75])
7.77
Params([[0.19925155; 0.19972123;;; 0.1982575; 0.19943172], [0.19925155; 0.19972123;;; 0.1982575; 0.19943172]])
(a = [-1.4022453; -3.4008362;;; -2.017425; -6.005683], b = [-0.70112264; -1.7004181;;; -0.50435627; -1.5014207])
5.891634
Params([[0.2970295; 0.29894242;;; 0.29260504; 0.29777992], [0.2970295; 0.29894242;;; 0.29260504; 0.29777992]])
(a = [-1.1089115; -3.1031728;;; -1.0739496; -5.0222006], b = [-0.55445576; -1.5515864;;; -0.2684874; -1.2555501])
4.363333
Params([[0.39238304; 0.39741224;;; 0.37976182; 0.39439976], [0.39238304; 0.39741224;;; 0.37976182; 0.39439976]])
(a = [-0.82285094; -2.8077633;;; -0.20238185; -4.0560026], b = [-0.41142547; -1.4038817;;; -0.050595462; -1.0140007])
3.1709118
Params([[0.4840957; 0.49484667;;; 0.4553892; 0.4884957], [0.4840957; 0.49484667;;; 0.455389

In [13]:
a = rand(2, 1, 2)
b = rand(1, 1, 2)
println(b)
println(size(NNlib.batched_mul(a, b)))

[0.7043097853266815

;;; 0.4922288426770137]
(2, 1, 2)


In [36]:
function test_mlp(prop_method)
    small_nnet_file = "/home/verification/ModelVerification.jl/test/networks/small_nnet.nnet"
    # small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
    small_nnet = read_nnet(small_nnet_file, last_layer_activation = ModelVerification.ReLU())
    flux_model = Flux.Chain(small_nnet)
    #ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/small_nnet.onnx", flux_model, (1,1))
    #println(flux_model)
    #println(flux_model.layers[1].weight, " ", flux_model.layers[1].bias) # max(x+1.5, 0) max(x+1.5, 0)              [0,4]
    #println(flux_model.layers[2].weight, " ", flux_model.layers[2].bias) # 4*max(x+1.5, 0)+2.5 4*max(x+1.5, 0)+2.5  [2.5, 18.5]
    #println(flux_model.layers[3].weight, " ", flux_model.layers[3].bias) # 24*max(x+1.5, 0)+18.5                    [18.5, 114.5]
    in_hyper  = Hyperrectangle(low = [-2.5], high = [2.5]) # expected out: [18.5, 114.5]
    out_violated    = Hyperrectangle(low = [19])#, high = [114]) # 20.0 ≤ y ≤ 90.0
    out_holds = Hyperrectangle(low = [18], high = [115.0]) # -1.0 ≤ y ≤ 50.0
    comp_violated    = Complement(Hyperrectangle(low = [10], high = [19])) # y ≤ 10.0 or 19 ≤ y
    comp_holds    = Complement(Hyperrectangle(low = [115], high = [118])) # y ≤ 10.0 or 18 ≤ y
    info = nothing
    search_method = BFS(max_iter=3, batch_size=2)
    split_method = BaBSR(2)
    #@test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_violated)).status == :violated
    #@test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_holds)).status == :holds
    #@test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_violated)).status == :violated
end
@timed begin
    #for i in 1:1
        test_mlp(BetaCrown(false, 2, Crown(false, true, true), true, false, Flux.Optimiser(Flux.ADAM(0.1)), 3))
        #test_mlp(Ai2z())
        #test_mlp(Crown(true, true))
        #test_mlp(StarSet(Crown(true, true)))
    #end
end


AssertionError: AssertionError: radius must not be negative

In [22]:
function test_mlp(prop_method)
    small_nnet_file = "/home/verification/ModelVerification.jl/test/networks/small_nnet.nnet"
    # small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
    small_nnet = read_nnet(small_nnet_file, last_layer_activation = ModelVerification.ReLU())
    flux_model = Flux.Chain(small_nnet)
    #ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/small_nnet.onnx", flux_model, (1,1))
    #println(flux_model)
    #println(flux_model.layers[1].weight, " ", flux_model.layers[1].bias) # max(x+1.5, 0) max(x+1.5, 0)              [0,4]
    #println(flux_model.layers[2].weight, " ", flux_model.layers[2].bias) # 4*max(x+1.5, 0)+2.5 4*max(x+1.5, 0)+2.5  [2.5, 18.5]
    #println(flux_model.layers[3].weight, " ", flux_model.layers[3].bias) # 24*max(x+1.5, 0)+18.5                    [18.5, 114.5]
    in_hyper  = Hyperrectangle(low = [-2.5], high = [2.5]) # expected out: [18.5, 114.5]
    out_violated    = Hyperrectangle(low = [19], high = [114]) # 20.0 ≤ y ≤ 90.0
    out_holds = Hyperrectangle(low = [18], high = [115.0]) # -1.0 ≤ y ≤ 50.0
    comp_violated    = Complement(Hyperrectangle(low = [10], high = [19])) # y ≤ 10.0 or 19 ≤ y
    comp_holds    = Complement(Hyperrectangle(low = [115], high = [118])) # y ≤ 10.0 or 18 ≤ y
    info = nothing
    search_method = BFS(max_iter=3, batch_size=2)
    split_method = Bisect(1)
    #@test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_violated)).status == :violated
    #@test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_holds)).status == :holds
    #@test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_violated)).status == :violated
    #= @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_violated)).status == :violated
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_violated)).status == :violated =#
end
@timed begin
    #for i in 1:1
        test_mlp(AlphaCrown(false, Crown(false, true, true), true, false, Flux.Optimiser(Flux.ADAM(0.1)), 3))
        #test_mlp(Ai2z())
        #test_mlp(Crown(true, true))
        #test_mlp(StarSet(Crown(true, true)))
    #end
end

start
1
Any[Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([0.0], [2.5])]
relu_3
[0.0;;]
relu_2
[0.0; 0.0;;]
relu_1
[0.7000000002500008; 0.7000000002500008;;]
2
3
Any[Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([-1.25], [1.25]), Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([1.25], [1.25])]
relu_3
[0.0 0.0]
relu_2
[0.0 0.0; 0.0 0.0]
relu_1
[0.7000000002500008 0.0; 0.7000000002500008 0.0]
Test Failed at /home/verification/ModelVerification.jl/tmp/test.ipynb:20
  Expression: (verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_violated))).status == :violated
   Evaluated: unknown == violated


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [14]:
a = [0.5284647839096102 0.9466078796040489; 0.2079903093502885 0.19692774736051; 0.8221945547831693 0.3318526763622769]
b = [0.5581055353150743 0.9204138604104602 0.49169644895013387 0.7841737348642372 0.3046794102208209; 0.3340520423890394 0.14292402232326062 0.5819823567683335 0.4363631211893606 0.21381568166361997]
c = [0.19615167803663092; 0.4362831955997829; 0.7907563085565711; 0.5431047673653794;;]
score = [a, b, c]
vec_score = []
relu_node_neurons_range = []
split_neurons_in_node = []
split_neurons_index_in_node = []
k = 10
current_neuron_index = 1
for matrix in score # matrix store neurons
    vec_matrix = vec(matrix)# all neurons need to be flattened into a vector 
    vec_score = vcat(vec_score, vec_matrix)
    push!(relu_node_neurons_range, [current_neuron_index, current_neuron_index + length(vec_matrix) - 1])
    current_neuron_index += length(vec_matrix)
end
topk_index = partialsortperm(vec_score, 1:k, rev = true)
topk_index = sort!(topk_index)
println(topk_index)
current_relu_node_neurons_range = relu_node_neurons_range[2]
current_relu_node_split_neurons_index = topk_index[(topk_index .>= current_relu_node_neurons_range[1]) .& (topk_index .<= current_relu_node_neurons_range[2])]
split_neurons_index_in_node = current_relu_node_split_neurons_index .- current_relu_node_neurons_range[1] .+ 1
println(split_neurons_index_in_node)
println(vec_score[current_relu_node_split_neurons_index])

split_neurons_in_node = zeros(size(vec(b)))
split_neurons_in_node[split_neurons_index_in_node] = b[split_neurons_index_in_node]
println(split_neurons_in_node)
println(size(split_neurons_index_in_node))
split_neurons_in_node = reshape(split_neurons_in_node, size(b))
println(size(b))
println(size(split_neurons_in_node))
println(split_neurons_in_node)

neg_split_neurons_in_node = ones(size(vec(b)))
neg_split_neurons_in_node[split_neurons_index_in_node] .= 0
neg_split_neurons_in_node = reshape(neg_split_neurons_in_node, size(b))
println(neg_split_neurons_in_node)
#splited_neurons_mask .*= split_neurons_in_node
#batch_info[node][splited_neurons_mask] = splited_neurons_mask

[1, 3, 4, 7, 9, 11, 12, 13, 19, 20]


[1, 3, 5, 6, 7]


Any[0.5581055353150743, 0.9204138604104602, 0.49169644895013387, 0.5819823567683335, 0.7841737348642372]


[0.5581055353150743, 0.0, 0.9204138604104602, 0.0, 0.49169644895013387, 0.5819823567683335, 0.7841737348642372, 0.0, 0.0, 0.0]
(

5,)


(2, 5)
(2, 5)


[0.5581055353150743 0.9204138604104602 0.49169644895013387 0.7841737348642372 0.0; 0.0 0.0 0.5819823567683335 0.0 0.0]


[0.0 0.0 0.0 0.0 1.0; 1.0 1.0 0.0 1.0 1.0]


In [35]:
function relu_upper_bound(lower, upper)
    lower_r = clamp.(lower, -Inf, 0)
    upper_r = clamp.(upper, 0, Inf)
    upper_r .= max.(upper_r, lower_r .+ 1e-8)
    upper_slope = upper_r ./ (upper_r .- lower_r) #the slope of the relu upper bound
    upper_bias = - lower_r .* upper_slope #the bias of the relu upper bound
    return upper_slope, upper_bias
end

lower = [0.0733 -0.2387; -0.4172 0.4561; 0.4147 -0.2835; 0.2980 -0.4698; -0.0005 0.2590; 0.2553 0.0367; -0.2080 -0.1935; -0.1256 0.3723; 0.4952 -0.0880; -0.1191 -0.2540]
A = [0.8017 0.7030 0.6828 0.1961 0.3994 0.3634 0.7698 0.8867 0.9313 0.5039;;; 0.1914 0.7184 0.8463 0.8017 0.1001 0.2627 0.8231 0.8939 0.3224 0.4989]
upper = lower .+ 0.2
unstable_mask = (upper .> 0) .& (lower .< 0)
unstable_mask = reshape(unstable_mask, (1, size(unstable_mask)...))
upper_slope, upper_bias = relu_upper_bound(lower, upper)
intercept_temp = clamp.(A, -Inf, 0)
intercept_candidate = intercept_temp .* reshape(upper_bias, (1, size(upper_bias)...))
b_temp = [0.1092, 0.3603, 0.0835, 0.1580, 0.2360, 0.6852, 0.2818, 0.0740, 0.6898, 0.1128]
b_temp = reshape(b_temp, (1, size(b_temp)...)) .* A
println(b_temp)
println(size(b_temp))
upper_slope = reshape(upper_slope, (1, size(upper_slope)...))
bias_candidate_1 = b_temp .* (upper_slope .- 1)
bias_candidate_2 = b_temp .* upper_slope
bias_candidate = max.(bias_candidate_1, bias_candidate_2)
score_candidate = bias_candidate .+ intercept_candidate
score_candidate = dropdims(mean((abs.(score_candidate) .* unstable_mask), dims = 1), dims = 1)
println(size(score_candidate))
println(score_candidate)

[0.08754564 0.2532909 0.0570138 0.0309838 0.09425839999999999 0.24900168 0.21692964 0.0656158 0.64241074 0.05683992;;; 0.02090088 0.25883952000000005 0.07066605000000001 0.1266686 0.023623599999999998 0.18000204 0.23194958000000002 0.0661486 0.22239152 0.05627592]
(1, 10, 2)
(10, 2)
[0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.094022754 0.0; 0.0 0.0; 0.0 0.007538361350000008; 0.02440907760000001 0.0; 0.0 0.12453925120000002; 0.02299174764 0.0]


In [ ]:
function relu_upper_bound(lower, upper)
    lower_r = clamp.(lower, -Inf, 0)
    upper_r = clamp.(upper, 0, Inf)
    upper_r .= max.(upper_r, lower_r .+ 1e-8)
    upper_slope = upper_r ./ (upper_r .- lower_r) #the slope of the relu upper bound
    upper_bias = - lower_r .* upper_slope #the bias of the relu upper bound
    return upper_slope, upper_bias
end

function branching_scores_kfsb(A, model_info, batch_info)
    score = []
    for node in reverse(model_info.activation_nodes)
        layer = model_info.node_layer[node]
        unstable_mask = batch_info[node][:unstable_mask]
        unstable_mask = reshape(unstable_mask, (1, size(unstable_mask)...))
        lower = batch_info[node][:pre_lower]
        upper = batch_info[node][:pre_upper]
        upper_slope, upper_bias = relu_upper_bound(lower, upper)

        intercept_temp = clamp.(A, -Inf, 0)
        intercept_candidate = intercept_temp .* reshape(upper_bias, (1, size(upper_bias)...))

        input_node = model_info.pre_layer[node][1]
        input_layer = model_info.node_layer[input_node]
        if isa(layer, Flux.Conv)
            if !isnothing(input_layer.bias)
                b_temp = input_layer.bias
            else
                b_temp = 0
            end
        elseif isa(layer, Flux.Dense)
            if !isnothing(input_layer.bias)
                b_temp = input_layer.bias
            else
                b_temp = 0
            end
        elseif isa(layer, +)
            b_temp = 0
            for l in model_info.pre_layer[input_node]
                l_layer = model_info.node_layer[l]
                if isa(layer, Flux.Conv)
                    if length(l_layer.inputs) > 2
                        b_temp += typeof.bias
                    end
                end
                if isa(layer, Flux.normalise)
                    b_temp += 0
                end
                if isa(layer, +)
                    for ll in model_info.pre_layer[l]
                        ll_layer = model_info.node_layer[ll]
                        if isa(layer, Flux.Conv)
                            b_temp += ll_layer.bias
                        end
                    end
                end
            end
        else
            b_temp = 0
        end
        
        b_temp = reshape(b_temp, (1, size(b_temp)...)) .* A
        bias_candidate_1 = b_temp .* (upper_slope .- 1)
        bias_candidate_2 = b_temp .* upper_slope
        bias_candidate = max.(bias_candidate_1, bias_candidate_2)
        score_candidate = bias_candidate .+ intercept_candidate
        score_candidate = dropdims(mean((abs.(score_candidate) .* unstable_mask), dims = 1), dims = 1)
        push!(score, score_candidate)
    end
    return score
end

In [12]:
a = [1.0; -1.0;;;]
println(size(a))
b = [18.5;;]
println(size(b))
c = batched_vec(a, b)
println(size(c))
d = batched_mul(a, b)
println(size(d))

(2, 1)
(2, 1, 1)


In [45]:
last_A = [1;;;]
println(size(last_A))
x = [18.5;;]
println(size(x))
bias = [0;;]
println(size(bias))
out = NNlib.batched_mul(last_A, x) .+ bias
println(size(out))
println(out)

(1, 1, 1)
(1, 1)
(1, 1)


(1, 1, 1)
[18.5;;;]


In [25]:
last_A = [9.6 9.6;;;]
bias = [1.5; 1.5;;]
println(bias)
New_bias = NNlib.batched_mul(last_A, bias)
println(size(New_bias))
println(New_bias)


[1.5; 1.5;;]
(1, 1, 1)
[28.799999999999997;;;]


In [ ]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(512, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
input_set = ImageConvexHull(image_seeds)
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
flux_model = model
image_shape = (32, 32, 3, 5)
println(image_seeds)

In [23]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(flux_model, input_set, output_set))

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, image_seeds, output_set))

In [ ]:
model = Chain([
    Flux.flatten,
    Dense(784, 200, relu),
    Dense(200, 10)
])
image_seeds = [MNIST(:train)[i].features for i in 1:1]
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/debug.onnx"
Flux_model = model
image_shape = (28, 28, 1, 1)

In [ ]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    Conv((3, 3), 8 => 8, relu, pad=SamePad(), stride=(2, 2)),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(128, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
Flux_model = model
image_shape = (32, 32, 3, 2)

In [ ]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

In [ ]:
model = Chain([
    Conv((3, 3), 3 => 128, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(128),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 128 => 128, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    ConvTranspose((3, 3), 128 => 128, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(32768, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

In [ ]:
Profile.clear()
@profile verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

In [ ]:
open("./prof.txt", "w") do s
    Profile.print(IOContext(s, :displaysize => (24, 500)))
end